

# Challenge Ingredion - Sprint 2
Este notebook foi desenvolvido para o Challenge em parceria com a Ingredion, na Sprint 2.

### Aplicação da Técnica de Segmentação - SAM



**Célula** 0: Montar Google Drive

In [ ]:
# Montar Google Drive (Necessário no Colab para acessar/salvar arquivos no Drive)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado com sucesso.")
    # Verificar se os diretórios de dados existem (opcional, mas útil)
except ModuleNotFoundError:
    print("Não está no ambiente Google Colab ou erro ao montar Drive.")
    # Se não estiver no Colab, ajuste os caminhos para serem locais

**Célula** 1: Instalação, Imports e Configurações

In [4]:
#!pip install torch torchvision albumentations opencv-python-headless matplotlib numpy pandas seaborn scikit-learn

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2 # Importante se usar ToTensorV2() nas transforms
from sklearn.model_selection import train_test_split # Para divisão treino/teste
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Configurações e Caminhos ---
DRIVE_BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data"

if not os.path.isdir(DRIVE_BASE_PATH):
  print(f"Aviso: Diretório base '{DRIVE_BASE_PATH}' não encontrado no Drive. Verifique os caminhos.")

IMAGE_TRAIN_DIR = os.path.join(DRIVE_BASE_PATH, "train/images")
MASK_TRAIN_DIR = os.path.join(DRIVE_BASE_PATH, "train/masks")
IMAGE_VAL_DIR = os.path.join(DRIVE_BASE_PATH, "val/images")
MASK_VAL_DIR = os.path.join(DRIVE_BASE_PATH, "val/masks")
IMAGE_TEST_DIR = os.path.join(DRIVE_BASE_PATH, "test/images")
MASK_TEST_DIR = os.path.join(DRIVE_BASE_PATH, "test/masks")

PROD_CSV_PATH = os.path.join(DRIVE_BASE_PATH, "Producao.csv")
print(f"Caminho PROD_CSV_PATH: {PROD_CSV_PATH}")
NDVI_CSV_PATH = os.path.join(DRIVE_BASE_PATH, "NDVI.csv")
print(f"Caminho NDVI_CSV_PATH: {NDVI_CSV_PATH}")

# Caminho para salvar o melhor modelo de segmentação
# Salvar no Drive é recomendado para persistência entre sessões do Colab
SAVE_DIR = "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output"
os.makedirs(SAVE_DIR, exist_ok=True) # Cria o diretório se não existir
BEST_MODEL_PATH = os.path.join(SAVE_DIR, 'best_segmentation_model.pth')

# Parâmetros de Treinamento (Segmentação)
NUM_EPOCHS_SEGMENTATION = 10 # AJUSTE (10 é baixo para resultados reais)
BATCH_SIZE_SEGMENTATION = 4
LEARNING_RATE_SEGMENTATION = 1e-4

# Parâmetros de Modelagem (Previsão de Rendimento)
RANDOM_STATE = 42
N_ESTIMATORS_RF = 100
TEST_YEARS_SPLIT = 10 # Número de anos recentes para usar como teste

# Configuração do dispositivo (GPU ou CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {DEVICE}")

Caminho PROD_CSV_PATH: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/Producao.csv
Caminho NDVI_CSV_PATH: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/NDVI.csv
Usando dispositivo: cpu


In [5]:
# Célula para Geração Dinâmica de Máscaras (Otsu com Verificação de Existência)

import cv2
import numpy as np
import os
import time

# --- Diretórios (Definidos na Célula 1) ---
# Certifique-se que estas variáveis da Célula 1 estão definidas corretamente:
# DRIVE_BASE_PATH, IMAGE_TRAIN_DIR, MASK_TRAIN_DIR, IMAGE_VAL_DIR, MASK_VAL_DIR, IMAGE_TEST_DIR, MASK_TEST_DIR

print("--- Iniciando Geração Dinâmica de Máscaras (Método de Otsu com Verificação) ---")

# Lista de pares de diretórios (entrada de imagens, saída de máscaras)
dir_pairs = [
    (IMAGE_TRAIN_DIR, MASK_TRAIN_DIR),
    (IMAGE_VAL_DIR, MASK_VAL_DIR),
    (IMAGE_TEST_DIR, MASK_TEST_DIR)
]

total_images_processed = 0
total_errors = 0
total_skipped = 0 # Contador para imagens puladas
start_time = time.time()

# Loop pelos pares de diretórios (train, val, test)
for img_dir, mask_dir in dir_pairs:
    print(f"\nProcessando diretório: {img_dir}")

    if not os.path.isdir(img_dir):
        print(f"Aviso: Diretório de entrada não encontrado: {img_dir}. Pulando.")
        continue

    try:
        os.makedirs(mask_dir, exist_ok=True)
    except OSError as e:
        print(f"Erro ao criar diretório de saída {mask_dir}: {e}. Pulando este diretório.")
        continue

    try:
        image_files = [f for f in os.listdir(img_dir) if f.lower().endswith('.jpg')]
        print(f"Encontradas {len(image_files)} imagens .jpg.")
    except Exception as e:
        print(f"Erro ao listar arquivos em {img_dir}: {e}. Pulando este diretório.")
        continue

    if not image_files:
        print("Nenhuma imagem .jpg encontrada para processar.")
        continue

    # Loop por cada imagem no diretório atual
    for img_filename in image_files:
        image_path = os.path.join(img_dir, img_filename)

        # --- INÍCIO DA VERIFICAÇÃO ---
        # Construir o nome e caminho esperado para a máscara de saída
        mask_filename = os.path.splitext(img_filename)[0] + "_mask.png"
        save_path = os.path.join(mask_dir, mask_filename)

        # Verificar se a máscara já existe
        if os.path.exists(save_path):
            # print(f"  Máscara já existe para {img_filename}. Pulando.") # Descomente se quiser log detalhado
            total_skipped += 1
            continue # Pula para a próxima imagem no loop interno
        # --- FIM DA VERIFICAÇÃO ---

        # Se a máscara não existe, processa a imagem
        try:
            img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img_gray is None:
                if not os.path.exists(image_path):
                   print(f"Erro: Arquivo não encontrado {image_path}. Pulando.")
                else:
                   print(f"Erro: Imagem não pode ser lida ou está vazia: {img_filename}. Pulando.")
                total_errors += 1
                continue

            # Aplicar Binarização de Otsu
            ret_otsu, mask_otsu = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Salvar a máscara gerada
            success = cv2.imwrite(save_path, mask_otsu)
            if not success:
                print(f"Erro ao salvar máscara para {img_filename} em {save_path}. Pulando.")
                total_errors += 1
                continue

            total_images_processed += 1
            if total_images_processed % 50 == 0:
                print(f"  {total_images_processed} novas máscaras geradas...")

        except Exception as e:
            print(f"Erro ao processar imagem {img_filename}: {e}. Pulando.")
            total_errors += 1

    print(f"Diretório {img_dir} concluído.")

end_time = time.time()
elapsed_time = end_time - start_time

print("\n--- Processamento Concluído ---")
print(f"Total de NOVAS máscaras geradas: {total_images_processed}")
print(f"Total de imagens PULADAS (máscara já existia): {total_skipped}")
print(f"Total de erros encontrados durante o processamento: {total_errors}")
print(f"Tempo total: {elapsed_time:.2f} segundos")
print(f"Máscaras salvas (ou verificadas) nos diretórios: {MASK_TRAIN_DIR}, {MASK_VAL_DIR}, {MASK_TEST_DIR}")

--- Iniciando Geração Dinâmica de Máscaras (Método de Otsu com Verificação) ---

Processando diretório: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/train/images
Encontradas 4 imagens .jpg.
Diretório /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/train/images concluído.

Processando diretório: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/val/images
Encontradas 3 imagens .jpg.
Diretório /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/val/images concluído.

Processando diretório: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/test/images
Encontradas 3 imagens .jpg.
Diretório /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/test/images concluído.

--- Processamento Concluído ---
Total de NOVAS máscaras geradas: 10
Total de imagens PULADAS (máscara já existia): 0
Total de erros encontrados durante o proces

Célula 2: Definição da Classe

In [6]:
# Classe Dataset para carregar imagens e máscaras de segmentação
class SatelliteDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        # Filtrar para garantir que estamos lendo apenas arquivos de imagem esperados (ex: .jpg)
        try:
          self.images = [f for f in os.listdir(image_dir) if f.lower().endswith('.jpg')]
        except FileNotFoundError:
          print(f"Erro: Diretório de imagens não encontrado: {image_dir}")
          self.images = []
          # Levantar erro ou tratar como apropriado
          # raise

        # Verifique se todos os arquivos têm máscaras correspondentes
        missing_masks = []
        valid_images = []
        if self.images: # Procede apenas se encontrou imagens
            for img_name in self.images:
                mask_name = img_name.replace(".jpg", "_mask.png") # Assumindo convenção _mask.png
                mask_path = os.path.join(mask_dir, mask_name)
                if os.path.exists(mask_path):
                    valid_images.append(img_name)
                else:
                    missing_masks.append(mask_path)

            if missing_masks:
                print(f"Aviso: Máscaras não encontradas para {len(missing_masks)} imagens em {mask_dir}.")
                # print(f"Exemplos de máscaras faltando: {missing_masks[:5]}") # Descomente para debug
            self.images = valid_images # Use only images with corresponding masks
        else:
            print(f"Nenhuma imagem .jpg encontrada em {image_dir}")


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if not self.images: return None, None # Lida com caso de diretório vazio

        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = img_name.replace(".jpg", "_mask.png")
        mask_path = os.path.join(self.mask_dir, mask_name)

        try:
            image = cv2.imread(img_path)
            if image is None:
                # Tentar carregar mesmo se nome tiver case diferente (comum em alguns sistemas)
                if not os.path.exists(img_path):
                   raise FileNotFoundError(f"Arquivo de imagem não encontrado: {img_path}")
                raise ValueError(f"Imagem não pode ser lida ou está vazia: {img_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                 # Tentar carregar mesmo se nome tiver case diferente
                 if not os.path.exists(mask_path):
                    raise FileNotFoundError(f"Arquivo de máscara não encontrado: {mask_path}")
                 raise ValueError(f"Máscara não pode ser lida ou está vazia: {mask_path}")

            # Normalizar máscara para 0 ou 1 (assumindo valores originais 0 e 255)
            mask = (mask > 128).astype(np.float32) # Convert to 0.0 or 1.0

            # Aplicar transformações
            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

            # Converter para Tensor PyTorch CHW (se a transform não o fez)
            # A.Normalize geralmente retorna numpy HWC; A.ToTensorV2 retorna tensor CHW
            if isinstance(image, np.ndarray):
                 # Normalização geralmente resulta em float32 HWC
                 image = torch.from_numpy(image).permute(2, 0, 1)
            if isinstance(mask, np.ndarray):
                 # Máscara é HW, converter para tensor float
                 mask = torch.from_numpy(mask).float()

        except Exception as e:
            print(f"Erro ao carregar item {idx} ({img_name}): {e}")
            return None, None # Retorna None para ser tratado pelo collate_fn

        return image, mask

print("Classe SatelliteDataset definida.")

Classe SatelliteDataset definida.


Célula 3: Transformações e DataLoaders (Segmentação)

In [7]:
# Transformações (usando médias/std da ImageNet)
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # Se não usar ToTensorV2, a conversão para Tensor e permuta é feita no __getitem__
])

val_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# Criar Datasets e DataLoaders
try:
    train_dataset = SatelliteDataset(IMAGE_TRAIN_DIR, MASK_TRAIN_DIR, train_transform)
    val_dataset = SatelliteDataset(IMAGE_VAL_DIR, MASK_VAL_DIR, val_transform)
    test_dataset = SatelliteDataset(IMAGE_TEST_DIR, MASK_TEST_DIR, val_transform)

    # Função collate para lidar com possíveis Nones retornados por __getitem__
    def collate_fn(batch):
        batch = list(filter(lambda x: x is not None and x[0] is not None, batch))
        if not batch: return None # Retorna None se o batch inteiro falhar
        return torch.utils.data.dataloader.default_collate(batch)

    # Verificar se os datasets foram criados com sucesso antes de criar DataLoaders
    if len(train_dataset) > 0:
       train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_SEGMENTATION, shuffle=True, num_workers=2, collate_fn=collate_fn, pin_memory=True)
       print(f"Train DataLoader criado com {len(train_dataset)} imagens.")
    else:
       train_loader = None
       print(f"Aviso: Train dataset vazio ou não pôde ser carregado de {IMAGE_TRAIN_DIR}. DataLoader de treino não criado.")

    if len(val_dataset) > 0:
       val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE_SEGMENTATION, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
       print(f"Validation DataLoader criado com {len(val_dataset)} imagens.")
    else:
       val_loader = None
       print(f"Aviso: Validation dataset vazio ou não pôde ser carregado de {IMAGE_VAL_DIR}. DataLoader de validação não criado.")

    if len(test_dataset) > 0:
      test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True) # Batch size 1 para teste/inferência
      print(f"Test DataLoader criado com {len(test_dataset)} imagens.")
    else:
      test_loader = None
      print(f"Aviso: Test dataset vazio ou não pôde ser carregado de {IMAGE_TEST_DIR}. DataLoader de teste não criado.")


except Exception as e:
    print(f"Erro ao criar Datasets/DataLoaders de segmentação: {e}")
    # Considerar parar a execução se os loaders não puderem ser criados
    # raise e

print("\nTransformações e DataLoaders definidos.")

Train DataLoader criado com 4 imagens.
Validation DataLoader criado com 3 imagens.
Test DataLoader criado com 3 imagens.

Transformações e DataLoaders definidos.


Célula 4: Definição do Modelo de Segmentação

In [8]:
# 1. Obtenha os pesos pré-treinados recomendados (DEFAULT)
weights = DeepLabV3_ResNet50_Weights.DEFAULT

# 2. Carregue o modelo DeepLabV3 pré-treinado usando o parâmetro 'weights'
model_segmentation = deeplabv3_resnet50(weights=weights)

# ----- Fim da Correção -----

# Modificar a última camada para 1 classe de saída (segmentação binária)
# A estrutura para acessar a camada final (classifier[4]) geralmente é estável,
# mas se encontrar erro aqui após atualizar, verifique a estrutura do modelo (print(model_segmentation))
model_segmentation.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

# Mover o modelo para o dispositivo (GPU ou CPU)
# Certifique-se que DEVICE está definido em alguma célula anterior
model_segmentation = model_segmentation.to(DEVICE)

# Definir Loss e Optimizer
criterion_segmentation = nn.BCEWithLogitsLoss() # Correto para saída binária com logits
# Certifique-se que LEARNING_RATE_SEGMENTATION está definido em alguma célula anterior
optimizer_segmentation = optim.Adam(model_segmentation.parameters(), lr=LEARNING_RATE_SEGMENTATION)

# Certifique-se que BEST_MODEL_PATH está definido em alguma célula anterior
print("Modelo de segmentação DeepLabV3+ definido com pesos atualizados e movido para o dispositivo.")
print(f"Caminho para salvar melhor modelo: {BEST_MODEL_PATH}")

Modelo de segmentação DeepLabV3+ definido com pesos atualizados e movido para o dispositivo.
Caminho para salvar melhor modelo: /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output/best_segmentation_model.pth


Célula 5: Função de Cálculo IoU (Segmentação)

In [9]:
# Função para calcular Intersection over Union (IoU)
def calculate_iou(pred, target, threshold=0.5):
    """Calcula IoU para segmentação binária usando tensores PyTorch."""
    # Garantir que os tensores estão na CPU e são do tipo float para operações
    pred = pred.detach().cpu().float()
    target = target.detach().cpu().float()

    pred_bool = (pred > threshold)
    target_bool = (target > 0.5) # Mascaras já devem ser 0 ou 1

    intersection = torch.logical_and(pred_bool, target_bool).sum().float()
    union = torch.logical_or(pred_bool, target_bool).sum().float()

    # Adicionar epsilon para evitar divisão por zero se a união for 0
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.item()

print("Função calculate_iou definida.")

Função calculate_iou definida.


Célula 6: Treinamento do Modelo de Segmentação

In [ ]:
# Verificar se o train_loader e val_loader foram criados
if train_loader and val_loader:
    print(f"\n--- Iniciando Treinamento ({NUM_EPOCHS_SEGMENTATION} épocas) ---")
    best_val_iou = -1.0 # Iniciar com valor baixo

    for epoch in range(NUM_EPOCHS_SEGMENTATION):
        model_segmentation.train()
        train_loss = 0.0
        train_iou = 0.0
        processed_batches_train = 0

        for batch in train_loader:
            if batch is None: continue # Pular batch inválido

            images, masks = batch
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            # Garantir que a máscara tenha o shape correto [B, H, W]
            if masks.dim() == 4 and masks.shape[1] == 1:
                 masks = masks.squeeze(1)

            # Forward pass
            outputs = model_segmentation(images)['out'] # Shape [B, 1, H, W]
            loss = criterion_segmentation(outputs.squeeze(1), masks) # Squeeze output para [B, H, W]

            # Backward pass e otimização
            optimizer_segmentation.zero_grad()
            loss.backward()
            optimizer_segmentation.step()

            train_loss += loss.item()
            preds = torch.sigmoid(outputs.squeeze(1))
            train_iou += calculate_iou(preds, masks) # Passar tensores diretamente
            processed_batches_train += 1

        avg_train_loss = train_loss / processed_batches_train if processed_batches_train > 0 else 0
        avg_train_iou = train_iou / processed_batches_train if processed_batches_train > 0 else 0

        # Validação
        model_segmentation.eval()
        val_loss = 0.0
        val_iou = 0.0
        processed_batches_val = 0
        with torch.no_grad():
            for batch in val_loader:
                if batch is None: continue
                images, masks = batch
                images, masks = images.to(DEVICE), masks.to(DEVICE)
                if masks.dim() == 4 and masks.shape[1] == 1:
                     masks = masks.squeeze(1)

                outputs = model_segmentation(images)['out']
                loss = criterion_segmentation(outputs.squeeze(1), masks)
                val_loss += loss.item()
                preds = torch.sigmoid(outputs.squeeze(1))
                val_iou += calculate_iou(preds, masks)
                processed_batches_val += 1

        avg_val_loss = val_loss / processed_batches_val if processed_batches_val > 0 else 0
        avg_val_iou = val_iou / processed_batches_val if processed_batches_val > 0 else 0

        print(f'Epoch {epoch+1}/{NUM_EPOCHS_SEGMENTATION} | '
              f'Train Loss: {avg_train_loss:.4f} | Train IoU: {avg_train_iou:.4f} | '
              f'Val Loss: {avg_val_loss:.4f} | Val IoU: {avg_val_iou:.4f}')

        # Salvar o melhor modelo
        if avg_val_iou > best_val_iou:
            best_val_iou = avg_val_iou
            try:
                torch.save(model_segmentation.state_dict(), BEST_MODEL_PATH)
                print(f'---> Modelo salvo em {BEST_MODEL_PATH} (Melhor Val IoU: {best_val_iou:.4f})')
            except Exception as e:
                print(f"Erro ao salvar o modelo em {BEST_MODEL_PATH}: {e}")

    print("\n--- Treinamento de segmentação concluído. ---")
else:
    print("\n--- Treinamento de segmentação pulado pois DataLoaders não foram criados corretamente. ---")


--- Iniciando Treinamento (10 épocas) ---
Epoch 1/10 | Train Loss: 0.6909 | Train IoU: 0.0003 | Val Loss: 0.6573 | Val IoU: 0.1470
---> Modelo salvo em /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output/best_segmentation_model.pth (Melhor Val IoU: 0.1470)
Epoch 2/10 | Train Loss: 0.6296 | Train IoU: 0.1084 | Val Loss: 0.6723 | Val IoU: 0.2544
---> Modelo salvo em /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output/best_segmentation_model.pth (Melhor Val IoU: 0.2544)
Epoch 3/10 | Train Loss: 0.6361 | Train IoU: 0.0859 | Val Loss: 0.6675 | Val IoU: 0.3178
---> Modelo salvo em /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output/best_segmentation_model.pth (Melhor Val IoU: 0.3178)
Epoch 4/10 | Train Loss: 0.5917 | Train IoU: 0.3167 | Val Loss: 0.6628 | Val IoU: 0.3327
---> Modelo salvo em /content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/output/best_segmentation_model.pth (Melhor Va

Célula 7: Avaliação do Modelo de Segmentação (Test Set)

In [ ]:
# Verificar se o test_loader existe e se o modelo foi salvo
if test_loader and os.path.exists(BEST_MODEL_PATH):
    print("\n--- Avaliando Melhor Modelo de Segmentação no Conjunto de Teste ---")
    try:
        # Carregar o melhor modelo salvo
        model_segmentation.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE)) # Usar map_location
        model_segmentation.eval()

        test_loss = 0.0
        test_iou = 0.0
        processed_batches_test = 0

        with torch.no_grad():
            for batch in test_loader:
                if batch is None: continue
                images, masks = batch
                images, masks = images.to(DEVICE), masks.to(DEVICE)
                if masks.dim() == 4 and masks.shape[1] == 1:
                    masks = masks.squeeze(1)

                outputs = model_segmentation(images)['out']
                # Tratamento para caso raro de output não ter a dimensão esperada
                if outputs.dim() == 4 and outputs.shape[1] == 1:
                   outputs_squeezed = outputs.squeeze(1)
                else:
                   # Lidar com caso inesperado, talvez pular o batch
                   print(f"Aviso: Shape de output inesperado: {outputs.shape}")
                   continue

                loss = criterion_segmentation(outputs_squeezed, masks)
                test_loss += loss.item()

                preds = torch.sigmoid(outputs_squeezed)
                test_iou += calculate_iou(preds, masks)
                processed_batches_test += 1

        if processed_batches_test > 0:
          avg_test_loss = test_loss / processed_batches_test
          avg_test_iou = test_iou / processed_batches_test
          print(f'Resultados no Teste: Loss: {avg_test_loss:.4f} | IoU: {avg_test_iou:.4f}')
        else:
            print("Nenhum batch de teste pôde ser processado.")

    except FileNotFoundError:
        print(f"Erro: Melhor modelo de segmentação não encontrado em {BEST_MODEL_PATH}.")
    except Exception as e:
        print(f"Erro durante a avaliação no teste de segmentação: {e}")
else:
    if not test_loader: print("\n--- Avaliação de teste pulada: Test DataLoader não disponível.")
    if not os.path.exists(BEST_MODEL_PATH): print(f"\n--- Avaliação de teste pulada: Modelo {BEST_MODEL_PATH} não encontrado.")

Célula 8: Função de Predição e Visualização (Segmentação)

In [ ]:
# Função para prever máscara de uma imagem individual
def predict_single_image(image_path, model, device, transform):
    """Prevê a máscara de segmentação para um único arquivo de imagem."""
    if not os.path.exists(image_path):
        print(f"Aviso: Arquivo não encontrado: {image_path}")
        return None, None # Retorna None se o arquivo não existe

    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Falha ao ler a imagem: {image_path}")
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Aplicar transformações de validação/teste
        augmented = transform(image=image_rgb)
        input_image_transformed = augmented['image']

        # Converter para tensor CHW
        if isinstance(input_image_transformed, np.ndarray):
            input_tensor = torch.from_numpy(input_image_transformed).permute(2, 0, 1).float()
        elif isinstance(input_image_transformed, torch.Tensor):
            input_tensor = input_image_transformed.float()
            if input_tensor.dim() == 3 and input_tensor.shape[2] == 3:
                 input_tensor = input_tensor.permute(2, 0, 1)
        else:
             raise TypeError("Tipo de saída inesperado da transformação.")

        input_tensor = input_tensor.unsqueeze(0).to(device)

        # Predição
        model.eval()
        with torch.no_grad():
            output = model(input_tensor)['out'].squeeze(0) # Remove B dim -> [C, H, W]

        # Processa a saída para máscara binária NumPy
        # Remove C dim (era 1), aplica sigmoid, threshold, move para CPU, numpy
        pred_mask = (torch.sigmoid(output.squeeze(0)) > 0.5).cpu().numpy()

    except Exception as e:
        print(f"Erro ao prever imagem {image_path}: {e}")
        return None, None

    return pred_mask, image_rgb

# Visualizar um exemplo do conjunto de teste (se disponível)
if test_dataset and len(test_dataset) > 0 and os.path.exists(BEST_MODEL_PATH):
    print("\n--- Visualizando Exemplo de Predição de Segmentação ---")
    try:
        # Carregar modelo (caso não esteja carregado)
        if 'model_segmentation' not in locals() or model_segmentation is None:
             model_segmentation = deeplabv3_resnet50(pretrained=False) # Não precisa pré-treinado aqui
             model_segmentation.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)
             model_segmentation.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
             model_segmentation = model_segmentation.to(DEVICE)
             model_segmentation.eval()

        # Obter item de teste
        item_idx = 0 # Pega o primeiro item
        test_image_name = test_dataset.images[item_idx]
        test_image_path = os.path.join(IMAGE_TEST_DIR, test_image_name)

        # Obter máscara real do dataset (já processada)
        _, mask_real_tensor = test_dataset[item_idx]
        if mask_real_tensor is not None:
           mask_real_np = mask_real_tensor.cpu().numpy() # Converte para numpy

           # Fazer a predição
           pred_mask_np, original_image_np = predict_single_image(
               test_image_path, model_segmentation, DEVICE, val_transform
           )

           if pred_mask_np is not None:
              # Calcular IoU para este exemplo
              iou_example = calculate_iou(torch.tensor(pred_mask_np), torch.tensor(mask_real_np))

              # Plot
              plt.figure(figsize=(15, 5))
              plt.subplot(1, 3, 1)
              plt.imshow(original_image_np)
              plt.title(f'Original ({test_image_name})')
              plt.axis('off')

              plt.subplot(1, 3, 2)
              plt.imshow(mask_real_np, cmap='gray')
              plt.title('Máscara Real')
              plt.axis('off')

              plt.subplot(1, 3, 3)
              plt.imshow(pred_mask_np, cmap='gray')
              plt.title(f'Predição (IoU: {iou_example:.2f})')
              plt.axis('off')
              plt.tight_layout()
              plt.show()
           else:
              print(f"Falha ao gerar predição para {test_image_path}")
        else:
            print(f"Falha ao obter máscara real para o item de teste {item_idx}")

    except IndexError:
        print("Índice fora do alcance no dataset de teste.")
    except Exception as e:
        print(f"Erro ao visualizar exemplo de segmentação: {e}")
else:
     print("\n--- Visualização de segmentação pulada (Dataset de teste ou modelo não disponível). ---")

Célula 9: Carregar e Pré-processar Dados Tabulares (Produtividade e NDVI)

In [ ]:
print("\n--- Carregando e Processando Dados Tabulares ---")

# --- Carregar Produtividade ---
df_prod = None # Inicializa para verificar carregamento
try:
    df_prod = pd.read_csv(PROD_CSV_PATH)
    print(f"Arquivo de produtividade '{PROD_CSV_PATH}' carregado.")
    # Selecionar e renomear colunas
    cols_prod_to_keep = {
        'Ano': 'Ano',
        'Área colhida (Hectares)': 'Area_Colhida_ha',
        'Rendimento médio da produção (Quilogramas por Hectare)': 'Rendimento_Medio_kg_ha'
    }
    # Verifica se todas as colunas ORIGINAIS existem
    if all(col in df_prod.columns for col in cols_prod_to_keep.keys()):
        df_prod = df_prod[list(cols_prod_to_keep.keys())].copy()
        df_prod.rename(columns=cols_prod_to_keep, inplace=True)
        df_prod['Ano'] = df_prod['Ano'].astype(int)
        print("Dados de produtividade selecionados e limpos.")
        print("Head df_prod:")
        print(df_prod.head())
    else:
        print(f"Aviso: Colunas esperadas ({list(cols_prod_to_keep.keys())}) não encontradas em {PROD_CSV_PATH}")
        print(f"Colunas encontradas: {df_prod.columns.tolist()}")
        df_prod = None # Anula se não encontrar colunas

except FileNotFoundError:
    print(f"Erro Crítico: Arquivo de produtividade '{PROD_CSV_PATH}' não encontrado.")
except Exception as e:
    print(f"Erro Crítico ao processar produtividade: {e}")

# --- Carregar e Limpar NDVI ---
df_ndvi = None # Inicializa
if df_prod is not None: # Só tenta carregar NDVI se produtividade carregou
    try:
        df_ndvi = pd.read_csv(NDVI_CSV_PATH, header=2) # Usa header=2 conforme descoberto
        if len(df_ndvi.columns) >= 2:
            # Renomeia as duas primeiras colunas e seleciona
            df_ndvi = df_ndvi.iloc[:, :2].copy() # Pega só as duas primeiras colunas
            df_ndvi.columns = ['Data', 'NDVI']
            # Remove a linha que contém 'Data', 'NDVI' como valores
            df_ndvi = df_ndvi[df_ndvi['Data'] != 'Data'].reset_index(drop=True)
        else:
            raise ValueError("CSV NDVI não tem colunas suficientes após carregar com header=2.")

        # Converter tipos
        df_ndvi['Data'] = pd.to_datetime(df_ndvi['Data'], format='%d/%m/%Y', errors='coerce')
        df_ndvi.dropna(subset=['Data'], inplace=True)
        if df_ndvi['NDVI'].dtype == 'object':
           df_ndvi['NDVI'] = df_ndvi['NDVI'].astype(str).str.replace(',', '.', regex=False)
        df_ndvi['NDVI'] = pd.to_numeric(df_ndvi['NDVI'], errors='coerce')
        df_ndvi.dropna(subset=['NDVI'], inplace=True)
        df_ndvi['Ano'] = df_ndvi['Data'].dt.year
        print(f"Arquivo NDVI '{NDVI_CSV_PATH}' carregado e limpo.")
        print("Head df_ndvi:")
        print(df_ndvi.head())

    except FileNotFoundError:
        print(f"Erro Crítico: Arquivo NDVI '{NDVI_CSV_PATH}' não encontrado.")
        df_ndvi = None # Garante que está None se falhar
    except Exception as e:
        print(f"Erro Crítico ao processar NDVI: {e}")
        df_ndvi = None # Garante que está None se falhar
else:
    print("Carregamento de NDVI pulado pois o arquivo de produtividade não foi carregado.")

Célula 10: Integração de Dados Tabulares e Engenharia de Features

In [ ]:
# Verificar se ambos DataFrames foram carregados
if df_prod is not None and df_ndvi is not None:
    print("\n--- Integrando Dados e Criando Features ---")

    # Agregar NDVI por Ano
    max_prod_year = df_prod['Ano'].max()
    df_ndvi_filtered = df_ndvi[df_ndvi['Ano'] <= max_prod_year].copy()
    ndvi_threshold = 0.1 # Filtro para remover ruídos
    df_ndvi_filtered_positive = df_ndvi_filtered[df_ndvi_filtered['NDVI'] > ndvi_threshold]

    if not df_ndvi_filtered_positive.empty:
        df_ndvi_agg = df_ndvi_filtered_positive.groupby('Ano')['NDVI'].agg(
            ndvi_medio='mean', ndvi_max='max', ndvi_min='min', ndvi_std='std', ndvi_count='count'
        ).reset_index()
        df_ndvi_agg['ndvi_std'] = df_ndvi_agg['ndvi_std'].fillna(0) # Preenche std=0 se só 1 valor no ano
        print("NDVI agregado por ano.")
        print("Head df_ndvi_agg:")
        print(df_ndvi_agg.head())

        # Merge
        df_final = pd.merge(df_prod, df_ndvi_agg, on='Ano', how='left')
        print("\nMerge entre produtividade e NDVI realizado.")

        # Adicionar Features Lagged
        df_final['Rendimento_Medio_kg_ha_lag1'] = df_final['Rendimento_Medio_kg_ha'].shift(1)
        df_final['Area_Colhida_ha_lag1'] = df_final['Area_Colhida_ha'].shift(1)
        print("Features lagged adicionadas.")

        # Limpeza final (remover NaNs do merge e do lag)
        df_final_cleaned = df_final.dropna().copy()
        print(f"\nDataFrame final pronto com {df_final_cleaned.shape[0]} linhas.")
        if not df_final_cleaned.empty:
             print(f"Cobertura Anual: {df_final_cleaned['Ano'].min()}-{df_final_cleaned['Ano'].max()}")
             print("Head df_final_cleaned:")
             print(df_final_cleaned.head())
             print("\nInfo df_final_cleaned:")
             df_final_cleaned.info()
        else:
            print("DataFrame final está vazio após limpeza (verificar merge e lags).")

    else:
        print("Não há dados de NDVI válidos (acima do threshold) para agregar. Impossível continuar.")
        df_final_cleaned = pd.DataFrame() # Cria dataframe vazio para evitar erros abaixo

else:
    print("\n--- Integração pulada: DataFrames df_prod ou df_ndvi não disponíveis. ---")
    df_final_cleaned = pd.DataFrame() # Cria dataframe vazio

Célula 11: Preparar Dados para Modelo de Rendimento e Dividir

In [ ]:
# Verificar se df_final_cleaned existe e não está vazio
if 'df_final_cleaned' in locals() and not df_final_cleaned.empty:
    print("\n--- Preparando Dados para Modelagem de Rendimento ---")
    # Definir Features (X) e Target (y)
    target = 'Rendimento_Medio_kg_ha'
    # Incluir todas as colunas exceto 'Ano' e o próprio target
    features = [col for col in df_final_cleaned.columns if col != target and col != 'Ano']

    # Garantir que o target e todas as features existem
    if target in df_final_cleaned.columns and all(f in df_final_cleaned.columns for f in features):
        X = df_final_cleaned[features]
        y = df_final_cleaned[target]
        print(f"Features (X) selecionadas: {features}")
        print(f"Target (y) selecionado: {target}")
        print(f"Shape de X: {X.shape}, Shape de y: {y.shape}")

        # Divisão Temporal Treino/Teste Manual
        split_index = len(X) - TEST_YEARS_SPLIT
        if split_index > 0 and TEST_YEARS_SPLIT > 0 :
            X_train, X_test = X[:split_index], X[split_index:]
            y_train, y_test = y[:split_index], y[split_index:]
            # Guardar os anos para visualização
            anos_train = df_final_cleaned['Ano'][:split_index].tolist()
            anos_test = df_final_cleaned['Ano'][split_index:].tolist()
            print(f"Divisão Temporal: Treino ({min(anos_train)}-{max(anos_train)}), Teste ({min(anos_test)}-{max(anos_test)})")
            print(f"Tamanho Treino: {X_train.shape[0]}, Tamanho Teste: {X_test.shape[0]}")
        else:
            print(f"Erro: Dados insuficientes ({len(X)} linhas) para criar conjunto de teste com {TEST_YEARS_SPLIT} anos.")
            # Resetar X_train, X_test etc para evitar erros na próxima célula
            X_train, X_test, y_train, y_test, anos_test = None, None, None, None, None
    else:
        print("Erro: Coluna target ou alguma coluna de feature não encontrada no DataFrame final.")
        X_train, X_test, y_train, y_test, anos_test = None, None, None, None, None

else:
    print("\n--- Preparação para modelagem pulada: df_final_cleaned não disponível ou vazio. ---")
    X_train, X_test, y_train, y_test, anos_test = None, None, None, None, None # Garante que não existem

Célula 12: Treinar Modelo de Previsão de Rendimento

In [ ]:
# Verificar se os dados de treino existem
if X_train is not None and y_train is not None:
    print("\n--- Treinando Modelo RandomForestRegressor para Rendimento ---")
    # Instanciar o modelo
    rf_model_yield = RandomForestRegressor(n_estimators=N_ESTIMATORS_RF,
                                         random_state=RANDOM_STATE,
                                         n_jobs=-1) # Usa todos os cores disponíveis

    # Treinar o modelo
    rf_model_yield.fit(X_train, y_train)
    print("Modelo de previsão de rendimento treinado com sucesso.")
else:
    print("\n--- Treinamento do modelo de rendimento pulado: Dados de treino não disponíveis. ---")
    rf_model_yield = None # Garante que modelo não existe

Célula 13: Avaliar e Visualizar Modelo de Rendimento

In [ ]:
# Verificar se o modelo foi treinado e se há dados de teste
if rf_model_yield is not None and X_test is not None and y_test is not None:
    print("\n--- Avaliando Modelo de Previsão de Rendimento ---")
    y_pred_yield = rf_model_yield.predict(X_test)

    # Calcular Métricas
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_yield))
    mae = mean_absolute_error(y_test, y_pred_yield)
    r2 = r2_score(y_test, y_pred_yield)

    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")
    print(f"R²:   {r2:.4f}")
    print("\nLembrete da Análise: O R² negativo indica desempenho ruim no teste.")
    print("O modelo atual não generaliza bem para os anos não vistos.")
    print("Considere as sugestões anteriores para melhoria ou documente as limitações.")


    # Visualizar Resultados (Previsto vs Real)
    print("\n--- Gerando Gráfico Previsto vs Real (Rendimento) ---")
    plt.figure(figsize=(12, 7)) # Aumentar tamanho
    plt.plot(anos_test, y_test, label='Real', marker='o', linewidth=2, markersize=8)
    plt.plot(anos_test, y_pred_yield, label='Previsto (RandomForest)', marker='x', linestyle='--', markersize=8)
    plt.xlabel('Ano', fontsize=12)
    plt.ylabel(f'{target} (kg/ha)', fontsize=12)
    plt.title('Previsão de Rendimento Médio - Conjunto de Teste (Anos {})'.format(f"{min(anos_test)}-{max(anos_test)}"), fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(anos_test, rotation=45) # Mostrar todos os anos no eixo x do teste
    plt.tight_layout() # Ajusta o layout para não cortar labels
    plt.show()

else:
    print("\n--- Avaliação/Visualização do modelo de rendimento pulada: Modelo ou dados de teste não disponíveis. ---")

Célula 14: Importância das Features (Modelo de Rendimento)

In [ ]:
# Verificar se o modelo e as features existem
if rf_model_yield is not None and 'features' in locals() and X_train is not None :
    try:
        print("\n--- Importância das Features (Modelo de Rendimento) ---")
        importances = rf_model_yield.feature_importances_
        # Criar dataframe de importância
        feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        print(feature_importance_df)

        # Plotar importância
        plt.figure(figsize=(10, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis', hue='Feature', legend=False)
        plt.title('Importância das Features - Modelo de Rendimento (RandomForest)', fontsize=14)
        plt.xlabel('Importância Relativa', fontsize=12)
        plt.ylabel('Feature', fontsize=12)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Não foi possível calcular/plotar a importância das features: {e}")
else:
    print("\n--- Cálculo da importância das features pulado: Modelo ou features não disponíveis. ---")

print("\n--- Fim do Notebook ---")